In [ ]:
HTML(read(open("style.css"), String))
# Pkg.add("NBInclude")
using NBInclude
# Pkg.add("Chess")
using Chess
using Test
using BenchmarkTools
using ProfileView
using Traceur
# using ProfileVega

In [ ]:
@nbinclude("EvaluatePosition.ipynb")

# Evaluate Position Testing

## Test CalculatePosition for a single piece



In [ ]:
startBoard=startboard()
queen = Piece(WHITE, QUEEN)
wPawn = Piece(WHITE, PAWN)
bPawn = Piece(BLACK, PAWN)
sD6 = Square(FILE_D, RANK_6)
sD2 = Square(FILE_D, RANK_2)
sD7 = Square(FILE_D, RANK_7)
b1 = fromfen("rnbqkbnr/ppp1pppp/8/3p4/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 1")
sD5 = Square(FILE_D, RANK_5)
@testset "Get Value of Piece" begin
    display(startBoard)
    @test calc_score(startBoard, queen, sD6) == 905
    @test calc_score(startBoard, wPawn, sD2) == 80 #100 pawn Value and -20 square 
    @test calc_score(startBoard, bPawn, sD7) == 80 #100 pawn Value and -20 square 
    display(b1)
    @test calc_score(b1, bPawn, sD5) == 120
end

## Evaluation of position 

In [ ]:
b1 = fromfen("rnbqkbnr/ppp1pppp/8/3P4/8/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1")
b2 = fromfen("rn1qkb1r/pppb1ppp/4pn2/3p4/3P1B2/2N5/PPPQPPPP/R3KBNR w KQkq - 2 5")
b3 = fromfen("1Q6/1R6/1B4k1/1N4p1/1P4n1/1K4b1/6r1/6q1 w - - 0 1")

@testset "Get Value of Piece" begin
    display(b1)
    @test evaluate_position(b1) == 125
    display(b2)
    @test evaluate_position(b2) == -5
    display(b3)
    @test evaluate_position(b3) == 0
end

## Test Capture Piece 

In [ ]:
sD5 = Square(FILE_D,RANK_5)
sA8 = Square(FILE_A,RANK_8)
b1 = fromfen("rnbqkbnr/ppp1pppp/8/3p4/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 1")
b2 = fromfen("rnbqkb1r/pP2pp1p/5np1/8/8/8/PPPP1PPP/RNBQKBNR w KQkq - 1 5")
@testset "Capture Piece" begin
    display(b1)
    @test valueCapturePiece(b1, sD5) == 120
    display(b2)
    @test valueCapturePiece(b2, sA8) == 500
end

## Test En Passant

In [ ]:
b1=fromfen("rnbqkbnr/ppp1pppp/8/3pP3/8/8/PPPP1PPP/RNBQKBNR b KQkq - 0 2")
domove!(b1, "f5")
sF6 = Square(FILE_F,RANK_6)
@testset "calcEnPassant" begin
    display(b1)
    @test calcEnPassant(b1, sF6) == 100
end

In [ ]:
b1=fromfen("rn1qkb1r/pppb1ppp/4pn2/3p4/3P1B2/2N5/PPPQPPPP/R3KBNR w KQkq - 2 5")
sC1 = Square(FILE_C,RANK_1)
piece = Piece(WHITE, KING)
@testset "calc_castle" begin
    display(board)
    @test calc_castle(b1, piece ,sC1) == 5
end

## Testing IsEndGame


In [ ]:
b1 = fromfen("2bqk3/8/8/8/8/8/5B2/3QK3 w - - 0 1")
b2 = fromfen("rnb1kbnr/ppp2ppp/8/8/8/8/PPP2PPP/RNB1KBNR w KQkq - 0 1")
b3 = fromfen("2b1kb2/pp4pp/8/8/8/8/PP4P1/R2QK3 w Q - 0 1")
b4 = fromfen("r1b1kbnr/p2p1ppp/2pP4/1p2p1B1/4P3/1B3P2/PPP2P1P/RN3RK1 w kq - 0 1")
@testset "Testing Endgame" begin
    display(b1)
    @test is_endgame(b1) == true
    display(b2)
    @test is_endgame(b2) == true
    display(b3)
    @test is_endgame(b3) == false
    display(b4)
    @test is_endgame(b4) == true
end

## Evaluate Move for terminal positions

In [ ]:
b1 = fromfen("7k/8/8/3K4/8/6R1/8/5R2 w - - 0 1")
m1 = movefromstring("f1h1")
b2 = fromfen("k7/7K/8/8/6r1/8/8/5r2 b - - 0 1")
m2 = movefromstring("f1h1")

b3 = fromfen("r2qkb1r/pp2np1p/3p1p2/2p1N1B1/2BnP3/3P4/PPP2PPP/R2bK2R w KQkq - 0 1")
m3 = movefromstring("c4f7")

@testset "Evaluate Move terminal position" begin
    display(b1)
    @test evaluate_move(b1, m1, evaluate_position(b1)) == evaluate_position(domove(b1, m1))
    display(b2)
    @test evaluate_move(b2, m2, evaluate_position(b2)) == evaluate_position(domove(b2, m2))
    display(b3)
    @test evaluate_move(b3, m3, evaluate_position(b3)) == evaluate_position(domove(b3, m3))
end

## Evaluate Move for endgame transition

In [ ]:
b1 = fromfen("r1b1kbnr/p2p1ppp/2pP4/1p2p1B1/4P3/1B3q2/PPP2PPP/RN3RK1 w kq - 0 1")
m1 = movefromstring("g2f3")
b2 = fromfen("1k6/1r6/1P2n3/6Pp/1R2p2P/4P3/P2NK3/3q4 w - - 0 1")
m2 = movefromstring("e2d1")
@testset "evaluate_move" begin
    display(b1)
    @test evaluate_move(b1, m1, evaluate_position(b1)) == evaluate_position(domove(b1, m1)) # regular position
    display(b2)
    @test evaluate_move(b2, m2, evaluate_position(b2)) == evaluate_position(domove(b2, m2)) # regular position
end

## Evaluate Move

In [ ]:
# white to move
b1 = startboard()
m1 = Move(Square(FILE_E,RANK_2), Square(FILE_E,RANK_4))

# Capture white and promotion
b2 = fromfen("rnbqkb1r/pP2pp1p/5np1/8/8/8/PPPP1PPP/RNBQKBNR w KQkq - 1 5")
m2 = Move(Square(FILE_B,RANK_7), Square(FILE_A,RANK_8), QUEEN)

# black to move
b3 = fromfen("7k/P7/8/8/8/8/8/K7 b - - 0 1")
m3 = movefromstring("h8h7")
b4 = fromfen("rnbqkbnr/ppp2ppp/8/3Pp3/8/5N2/PPPP1PPP/RNBQKB1R b KQkq - 0 1")
m4 = movefromstring("d8h4")

# Captures white
b5 = fromfen("rnb1kbnr/ppp2ppp/8/3Pp3/7q/5N2/PPPP1PPP/RNBQKB1R w KQkq - 0 1")
m5 = movefromstring("f3h4")

@testset "evaluate_move" begin
    display(domove(b1, m1))
    @test evaluate_move(b1, m1, evaluate_position(b1)) == evaluate_position(domove(b1, m1)) # regular position
    display(domove(b2, m2))
    @test evaluate_move(b2, m2, evaluate_position(b2)) == evaluate_position(domove(b2, m2))
    display(domove(b3, m3))
    @test evaluate_move(b3, m3, evaluate_position(b3)) == evaluate_position(domove(b3, m3)) # black king movement
    display(domove(b4, m4))
    @test evaluate_move(b4, m4, evaluate_position(b4)) == evaluate_position(domove(b4, m4)) # Queen blunder black
    display(domove(b5, m5))
    @test evaluate_move(b5, m5, evaluate_position(b5)) == evaluate_position(domove(b5, m5))
end

### Evaluate move - Promotion

In [ ]:
bw = fromfen("8/pk2P3/8/8/8/8/PK6/8 w - - 0 1")
mwN = movefromstring("e7e8N")    # mwN = move white kNight
mwB = movefromstring("e7e8B")
mwR = movefromstring("e7e8R")
mwQ = movefromstring("e7e8Q")

bb = fromfen("8/pk6/8/8/8/8/PK2p3/8 b - - 0 1")
mbN = movefromstring("e2e1N")
mbB = movefromstring("e2e1B")
mbR = movefromstring("e2e1R")
mbQ = movefromstring("e2e1Q")

@testset "evaluate_move_position" begin
    display(bw)
    @test evaluate_move(bw, mwN, evaluate_position(bw)) ==  evaluate_position(domove(bw, mwN)) # promotion white Knight
    @test evaluate_move(bw, mwB, evaluate_position(bw)) ==  evaluate_position(domove(bw, mwB)) # promotion white Bishop
    @test evaluate_move(bw, mwR, evaluate_position(bw)) ==  evaluate_position(domove(bw, mwR)) # promotion white Rook
    @test evaluate_move(bw, mwQ, evaluate_position(bw)) ==  evaluate_position(domove(bw, mwQ)) # promotion white Queen
    display(bb)
    @test evaluate_move(bb, mbN, evaluate_position(bb)) ==  evaluate_position(domove(bb, mbN)) # promotion black Knight
    @test evaluate_move(bb, mbB, evaluate_position(bb)) ==  evaluate_position(domove(bb, mbB)) # promotion black Bishop
    @test evaluate_move(bb, mbR, evaluate_position(bb)) ==  evaluate_position(domove(bb, mbR)) # promotion black Rook
    @test evaluate_move(bb, mbQ, evaluate_position(bb)) ==  evaluate_position(domove(bb, mbQ)) # promotion black Queen
end

## Testing with random move engine

In [ ]:
@nbinclude("RandomChess.ipynb")

In [ ]:
@testset "EvaluatePosition with Random Engine" begin
    Random.seed!(123)
    testBoard = startboard()
    testScore = 0
    while !isterminal(testBoard)
        move = generateRandomMove(testBoard)
        incrementalScore = evaluate_move(testBoard, move, testScore)
        undoInfo = domove!(testBoard, move)
        actualScore = evaluate_position(testBoard)
        @test incrementalScore == actualScore
        if incrementalScore != actualScore
            undomove!(testBoard, undoInfo)
            display(testBoard)
            println(move)
            domove!(testBoard, move)
        end
        testScore = actualScore
    end
end

Special case error from endgame to middlegame

In [ ]:
# is endgame
b = fromfen("1k1n4/1r6/1P6/4pBPp/7P/p3P3/PRp2K2/R4NN1 b - - 0 1")
display(b)
println(evaluate_position(b))
move = movefromstring("c2c1q")
println(evaluate_position(domove(b, move)))
evaluate_move(b, move, evaluate_position(b))

## Profiling

In [ ]:
b5 = fromfen("rnb1kbnr/ppp2ppp/8/3Pp3/7q/5N2/PPPP1PPP/RNBQKB1R w KQkq - 0 1")
m5 = movefromstring("f3h4")
display(b5)
score = evaluate_position(b5)
@profview begin
    for i in 1:100000
        evaluate_move(b5, m5, score)
    end
end

## Benchmarking

In [ ]:
b5 = fromfen("rnb1kbnr/ppp2ppp/8/3Pp3/7q/5N2/PPPP1PPP/RNBQKB1R w KQkq - 0 1")
m5 = movefromstring("f3h4")
display(b5)
score = evaluate_position(b5)
for i in 1:10
   @time evaluate_move(b5, m5, score)
end